In [ ]:
import os                                                             #for interacting with os for files 
import cv2                                                            #part of openCV for image processing tasks
import pandas as pd                                                   #handling and managing datas
import numpy as np                                                    #for numerical operations
import matplotlib.pyplot as plt                                       #used to create visualizations
import seaborn as sns                                                 #advance visulazation tools
from sklearn.metrics import confusion_matrix, classification_report   #for evaluating ml models by generating metrics
from sklearn.model_selection import train_test_split                  #splits the data for model evaluation and validation
import tensorflow as tf                                               #deeplearning framework for creating and training neural networks
from tensorflow.keras.models import Sequential                        #used to define a feed forward model by stacking layers sequentially
from tensorflow.keras.layers import Dropout                           #manipulates neurons in neural network
from tensorflow.keras.metrics import Recall, Precision                #metrics to evaluate model performance
from tensorflow.keras.preprocessing.image import ImageDataGenerator   #for agumentation
from tensorflow.keras.optimizers import Adam                          #adjusts the learning rate during training
from tensorflow.keras.applications import InceptionV3                 #pretrained deeplearing models for transfer learning
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Input,concatenate,Dense, GlobalAveragePooling2D, Flatten                #for input layer and merging multiple layers
from tensorflow.keras.models import Model, clone_model                #enables builiding complex model, allowing more flexible network
from tensorflow.keras.callbacks import EarlyStopping                  #monitors model performance and stops the training of model to prevent overfitting and saves time

In [ ]:
df = pd.read_csv("/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv")

In [ ]:
df.head()

In [ ]:
df.info()

Normal (N),
Diabetes (D),
Glaucoma (G),
Cataract (C),
Age related Macular Degeneration (A),
Hypertension (H),
Pathological Myopia (M),
Other diseases/abnormalities (O)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
sns.barplot(data=df, x="labels", y="Patient Age", hue="Patient Sex")

In [ ]:
if 'ID' in df.columns:
    df = df.drop('ID', axis=1)

In [ ]:
df.columns

In [ ]:
df['Patient Sex'].value_counts()

In [ ]:
import warnings

# Suppress specific warning
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    plt.figure(figsize=(6, 6))
    sns.histplot(data=df, x="Patient Age")


In [ ]:
plt.figure(figsize=(6,6))
plt.pie(df['labels'].value_counts(), autopct="%0.1F%%",
       labels=['Normal','Diabetes','Glaucoma','Cataract','Age related Macular Degeneration','Hypertension',
                'Pathological Myopia','Other diseases/abnormalities'],
       shadow=True, explode=[0.2, 0.09, 0,0,0,0,0,0])

In [ ]:
sex_counts = df['Patient Sex'].value_counts().reset_index()

sex_counts.columns = ['Sex', 'Count']

plt.figure(figsize=(6,6))
sns.barplot(x='Sex', y='Count', data=sex_counts, color='r', width=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.pie(df['Patient Sex'].value_counts(), autopct="%0.1F%%",
       labels=['Male', 'Female'], shadow=True, explode=[0.2,0])
plt.show()

In [ ]:
real_values = ["['N']", "['D']", "['O']", "['M']", "['H']", "['C']", "['A']",
       "['G']"]
df['labels'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder   #coverts non numerical value to numerical
encoder = LabelEncoder()
df['labels'] = encoder.fit_transform(df['labels'])

In [ ]:
df['labels'] = df['labels'].astype(str)

In [ ]:
left_images = df[['Left-Fundus','labels']]
Right_images = df[['Right-Fundus', 'labels']]

left_images.columns = ['image', 'target']
Right_images.columns = ['image', 'target']

left_images

In [ ]:
combined_df = pd.concat([left_images, Right_images])
combined_df = combined_df.dropna(subset='image')
combined_df

In [ ]:
train_df,test_df = train_test_split(combined_df, test_size=0.2, random_state=42)
train_df.head()

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = '/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images',
    x_col = 'image',
    y_col = 'target',
    subest = 'training',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = 16
)

In [ ]:
validation_generator = datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = '/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images',
    x_col = 'image',
    y_col = 'target',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = 32
)

!pip install -U efficientnet          

**installed efficientnet locally because the kernel was not able to import efficientnet from tensorflow.keras.applications**

In [ ]:
input_layer = Input(shape=(224,224,3))        #edit this cell

inceptionv3 = InceptionV3(weights='imagenet', include_top=False)(input_layer)
gop = GlobalAveragePooling2D()(inceptionv3)
                                                                       

dense1 = Dense(512, activation='relu')(gop)    #why 4 dense layers and why relu activation
dense2 = Dense(256, activation='relu')(dense1)
dense3 = Dense(128, activation='relu')(dense2)    #decrease dense layer and add (mukul dai)
dense4 = Dense(64, activation='relu')(dense3)

output_layer = Dense(8, activation='sigmoid')(dense4)  #visualize this and why softmax here?

model = Model(inputs = input_layer, outputs = output_layer)

In [ ]:
model.compile(optimizer= Adam(learning_rate=0.0001),                 #understand everthing from this
             loss='binary_crossentropy',
             metrics=[
                 Recall(name="recall"),
                 Precision(name="Precision"),
                 "accuracy"
             ])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

In [ ]:
# checkpoint_filepath = '/kaggle/working/checkpoint.weights.h5'    
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath = checkpoint_filepath,
#     save_weights_only = True,
#     monitor = 'val_loss',
#     mode='min',
#     save_best_only=True)
# )
# history = model.fit(train_generator, epochs=10, validation_data=validation_data-generator, callbacks=[model_checkpoint_callback])
# history = model.fit(train_generator, epochs=20, validation_data=validation_generator, callback = [early_stopping])

history = model.fit(train_generator, epochs=20, validation_data=validation_generator)